In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# own Modules 
from models import LstmMse
from data_loader import DataPreperator, DataSet
from trainer import Trainer
from loss_module import LossMse

## Hyperparameters

In [9]:
param = {
    "data" : {
        "stake_training_data" : 0.75, 
        "path" : '../../../data/phm_data_challenge/01_M01_DC_grid_search.csv' ,
    },
    "preprocessing" : {
        "first_order_difference" : False,
        "droped_features": ["ID", "stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "input_size" : 13,
        "n_hidden_lstm" : [50, 100, 200],
        "sequence_size" : [50, 100, 200],
        "batch_size" : 8,
        "lstm_layer" : [2, 4],
        "n_hidden_fc": [25, 50, 100],
        "dropout_rate": 0.2
        
    },
    "cycling_lr" : {
        "scheduler_active" : True, 
        # step_size is the number of training iterations (total samples/batch_size) per half cycle. 
        # Authors suggest setting step_size 2-8 x training iterations in epoch.
        "step_size" : (6000/8)*2,
        # Mode can be one of {triangular, triangular2, exp_range}
        "mode" : "triangular",
        "gamma" : 0.9995,
        "base_lr" : 0.016, 
        "max_lr" :0.07
    },
    "training": {
        "n_epochs" : 100,
        "patience" : 6,
    },
    "filed_location": {
        "trained_model" : "../../../models/MSE_model/grid_search/best_model",
        "statistics" : "../../visualisation/files/history_training/grid_search/statistics_grid_search.csv"
    }
}

## Split raw data into train and validation data and scale it

In [10]:
train_loader = DataPreperator(path=param['data']['path'], 
                              ignored_features=param['preprocessing']['droped_features'],
                              stake_training_data=param['data']['stake_training_data'],
                              features_not_to_scale=param['preprocessing']['features_not_to_scale'],
                              first_order_difference=param["preprocessing"]["first_order_difference"])
train_data, validation_data = train_loader.prepare_data()
print(len(train_data)+len(validation_data))

8000


## Grid Search
An epoch consists of a learning cycle over all batches of training data and an evaluation of the most recent model with the testing data. 

In [13]:
for n_lstm_layer in param["model"]["lstm_layer"]:
    for sequence_size  in param["model"]["sequence_size"]:
        # Create roling dataset 
        dataset_train = DataSet(train_data, timesteps=sequence_size)
        dataset_validation = DataSet(validation_data, timesteps=sequence_size)
    
        # Initialize DataLoader
        data_loader_training = DataLoader(dataset_train, 
                                          batch_size=param["model"]["batch_size"], 
                                          num_workers=0, 
                                          shuffle=True, 
                                          drop_last=True)
        data_loader_validation = DataLoader(dataset_validation, 
                                            batch_size=param["model"]["batch_size"], 
                                            num_workers=0, 
                                            shuffle=True, 
                                            drop_last=True)
        for n_hidden_lstm in param["model"]["n_hidden_lstm"]:
            for n_hidden_fc in param["model"]["n_hidden_fc"]:
                print("Start with new hyperparameters in grid search: ")
                print("Sequence_size: {}".format(sequence_size))
                print("Number LSTM Layers: {}".format(n_lstm_layer))
                print("LSTM Number Hidden Dimensions: {}".format(n_hidden_lstm))
                print("FC NN Number Hidden Dimensions: {}".format(n_hidden_fc))

                # Create lists to save training loss and validation loss of each epoch
                hist_loss = []
                torch.manual_seed(0)
                model = LstmMse(batch_size=param['model']['batch_size'], 
                                input_dim=param['model']['input_size'], 
                                n_hidden_lstm=n_hidden_lstm, 
                                n_layers=n_lstm_layer,
                                dropout_rate= param['model']['dropout_rate'],
                                n_hidden_fc=n_hidden_fc
                               )

                optimizer = torch.optim.SGD(model.parameters(), lr=1.)  

                criterion = LossMse(param["model"]["input_size"], param["model"]["batch_size"])

                scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer=optimizer, 
                                                              base_lr=param['cycling_lr']['base_lr'], 
                                                              max_lr=param['cycling_lr']['max_lr'], 
                                                              step_size_up=param['cycling_lr']['step_size'], 
                                                              mode=param['cycling_lr']['mode'],
                                                              gamma=param['cycling_lr']['gamma']
                                                             )

                trainer = Trainer(model=model,
                                  optimizer=optimizer,
                                  scheduler=scheduler,
                                  scheduler_active = param["cycling_lr"]["scheduler_active"],
                                  criterion=criterion, 
                                  location_model=param["filed_location"]["trained_model"], 
                                  location_stats=param["filed_location"]["statistics"], 
                                  patience=param['training']['patience']
                                 )
                
                # Measure training time for current configuration
                start = time.time()

                for epoch in range(param['training']['n_epochs']):
                    # Train
                    mean_epoch_training_loss = trainer.train(data_loader_training)

                    # Evaluate
                    mean_epoch_validation_loss = trainer.evaluate(data_loader_validation, hist_loss, epoch)

                    # Cache History
                    trainer.cache_history_training(hist_loss, epoch, mean_epoch_training_loss, mean_epoch_validation_loss)

                    # Save model if its the best one since the last change in configuration of hyperparameters
                    status_ok = trainer.save_model(epoch, mean_epoch_validation_loss, param['model']['input_size'], 
                                                   n_lstm_layer, n_hidden_lstm, n_hidden_fc, sequence_size)
                    if not status_ok:
                        break

                # Time in minutes
                execution_time = (time.time() - start)/60
                
                # Save training statistics 
                trainer.save_statistic(hist_loss, sequence_size, n_lstm_layer, n_hidden_lstm, n_hidden_fc, execution_time)         

Start with new hyperparameters in grid search: 
Sequence_size: 50
Number LSTM Layers: 2
LSTM Number Hidden Dimensions: 50
FC NN Number Hidden Dimensions: 25
-------- epoch_no. 0 finished with eval loss 0.11824164345645856--------
Epoch 0: best model saved with loss: 0.11824164345645856


KeyboardInterrupt: 